In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from shapely.geometry import Point, Polygon
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import networkx as nx
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv,SAGEConv,global_add_pool,global_mean_pool,global_max_pool
import numpy as np
from sklearn.model_selection import train_test_split
from torch_geometric.utils import to_networkx
import networkx as nx
from networkx.algorithms import community
from torch_geometric.data import Data
import community as community_louvain
from torch_geometric.loader import DataLoader
import random
import torch.nn as nn
import math
import os
from torch.utils.data import TensorDataset
from scipy.interpolate import interp1d
from torch.utils.tensorboard import SummaryWriter
import statsmodels.api as sm
from scipy.fftpack import fft, ifft
from sklearn.metrics import mean_squared_error
from math import exp
from fastdtw import fastdtw
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN, MeanShift
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import warnings
from sklearn.cluster import OPTICS
from sklearn.metrics import pairwise_distances
import seaborn as sns
import mplcursors
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap



def interpolate_by_depth(one_well,depth):
    
    one_well = one_well.sort_values(by=depth)
    well_name = one_well["well"].iloc[0]
    
    formation_name = one_well["FORMATION"].iloc[0]
    formation_up_name = one_well["FORMATION_up"].iloc[0]
    

    data_range = np.floor((one_well[depth].max() - one_well[depth].min())/0.1)
    starting_tvd = one_well[depth].iloc[0]
    
    new_depth_values = [starting_tvd + i*0.1 for i in range(1,int(data_range))]
    
    interp_X = interp1d(one_well[depth], one_well['X'], kind='linear', fill_value="extrapolate")
    interp_Y = interp1d(one_well[depth], one_well['Y'], kind='linear', fill_value="extrapolate")
    interp_PHIT = interp1d(one_well[depth], one_well['PHIT'], kind='linear', fill_value="extrapolate")
    interp_VSH = interp1d(one_well[depth], one_well['VSH'], kind='linear', fill_value="extrapolate")
    interp_GR = interp1d(one_well[depth], one_well['GR_N'], kind='linear', fill_value="extrapolate")
    interp_LPERM = interp1d(one_well[depth], one_well['LPERM'], kind='linear', fill_value="extrapolate")
    interp_KHtst = interp1d(one_well[depth], one_well['KHtst'], kind='linear', fill_value="extrapolate")
    interp_NET = interp1d(one_well[depth], one_well['NET_clp2'], kind='linear', fill_value="extrapolate")


    interpolated_NET = interp_NET(new_depth_values)
    
    
    interpolated_NET = np.where(interpolated_NET >= 0.5, 1, 0)
    # Create a new DataFrame with the interpolated values for new TVD_SCS
    new_data = {
        'well': [well_name for _ in range(len(new_depth_values))],
        depth: new_depth_values,
        'X': interp_X(new_depth_values),
        'Y': interp_Y(new_depth_values),
        'PHIT': interp_PHIT(new_depth_values),
        'VSH': interp_VSH(new_depth_values),
        'GR_N':interp_GR(new_depth_values),
        'LPERM':interp_LPERM(new_depth_values),
        'KHtst':interp_KHtst(new_depth_values),
        'NET_clp2':interpolated_NET
    }

    new_df = pd.DataFrame(new_data)
    new_df["FORMATION"] = formation_name
    new_df["FORMATION_up"] = formation_up_name
        
    return new_df



def create_directory(parent_dir, new_dir_name):
    """
    Create a new directory under the specified parent directory.

    Parameters:
    - parent_dir (str): The path to the parent directory.
    - new_dir_name (str): The name of the new directory to be created.

    Returns:
    - str: The full path of the newly created directory.
    """
    # Join the parent directory path with the new directory name
    new_dir_path = os.path.join(parent_dir, new_dir_name)

    try:
        # Create the directory if it does not exist
        os.makedirs(new_dir_path)
        print(f"Directory '{new_dir_name}' created successfully at '{parent_dir}'.")
        return new_dir_path
    except FileExistsError:
        print(f"Directory '{new_dir_name}' already exists at '{parent_dir}'.")
        return new_dir_path

def getPerm(PHI,tops):
    PERM = None
    
    if PHI >=0.20: # EQ5
        if  tops == 'Pereriv_D' or tops == 'Pereriv D':
            PERM = (6.7356*((PHI*100)**2))-(19889.0*PHI)+1433.1
        elif  tops == 'Pereriv_D_lower' or tops == 'Pereriv D lower':
            PERM = (6.7356*((PHI*100)**2))-(19889.0*PHI)+1433.1	
        elif tops == 'Pereriv_E' or tops == 'Pereriv E':
            PERM = (6.7356*((PHI*100)**2))-(19889.0*PHI)+1433.1
        elif tops == 'Pereriv_A' or tops == 'Pereriv A':
            PERM = (9.4372*((PHI*100)**2))-(32902.0*PHI)+2929.7
        elif tops == 'Pereriv_B' or tops == 'Pereriv B':
            PERM = (9.4372*((PHI*100)**2))-(32902.0*PHI)+2929.7
        elif tops == 'NKG':
            PERM = (9.4372*((PHI*100)**2))-(32902.0*PHI)+2929.7
        elif tops == 'Pereriv_C' or tops == 'Pereriv C':
            PERM = (4.6201*((PHI*100)**2))-(14661.0*PHI)+1140
        elif tops == 'Balakhany_VIII' or tops == 'Balakhany VIII':
            PERM = (7.7925*((PHI*100)**2))-(29881.0*PHI)+2891.8
        elif tops == 'Balakhany_VIII_sand' or tops == 'Balakhany VIII sand':
            PERM = (7.7925*((PHI*100)**2))-(29881.0*PHI)+2891.8
        else:
            # PERM = (4.6201*((PHI*100)**2))-(14661.0*PHI)+1140 Old balX transform,
            PERM = (7.7925*((PHI*100)**2))-(29881.0*PHI)+2891.8
    elif PHI >= 0.19 and PHI < 0.20:  # EQ4
            if  tops == 'Pereriv_D' or tops == 'Pereriv D':
                PERM = (6.7356*((PHI*100)**2))-(19889.0*PHI)+1433.1
            elif  tops == 'Pereriv_D_lower' or tops == 'Pereriv D lower':
                PERM = (6.7356*((PHI*100)**2))-(19889.0*PHI)+1433.1
            elif tops == 'Pereriv_E' or tops == 'Pereriv E':
                PERM = (6.7356*((PHI*100)**2))-(19889.0*PHI)+1433.1
            elif tops == 'Pereriv_A' or tops == 'Pereriv A':
                PERM = (9.4372*((PHI*100)**2))-(32902.0*PHI)+2929.7
            elif tops == 'Pereriv_B' or tops == 'Pereriv B':
                PERM = (9.4372*((PHI*100)**2))-(32902.0*PHI)+2929.7
            elif tops == 'NKG':
                PERM = (9.4372*((PHI*100)**2))-(32902.0*PHI)+2929.7
            elif tops == 'Pereriv_C' or tops == 'Pereriv C':
                PERM = (4.6201*((PHI*100)**2))-(14661.0*PHI)+1140
            elif tops == 'Balakhany_VIII' or tops == 'Balakhany VIII':
                PERM = 0.00000002*(exp(PHI*105.56))
            elif tops == 'Balakhany_VIII_sand' or tops == 'Balakhany VIII sand':
                PERM = 0.00000002*(exp(PHI*105.56))
            else:
                #PERM = (4.6201*((PHI*100)**2))-(14661.0*PHI)+1140 Old balX transform,
                PERM = 0.00000002*(exp(PHI*105.56))
                
    elif PHI >= 0.16 and PHI < 0.19:#EQ3
        if  tops == 'Pereriv_D' or tops == 'Pereriv D':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif  tops == 'Pereriv_D_lower' or tops == 'Pereriv D lower':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'Pereriv_E' or tops == 'Pereriv E':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'Pereriv_A' or tops == 'Pereriv A':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'Pereriv_B' or tops == 'Pereriv B':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'NKG':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'Pereriv_C' or tops == 'Pereriv C':
            PERM = 0.00000006*(exp(PHI*103.44))
        elif tops == 'Balakhany_VIII' or tops == 'Balakhany VIII':
            PERM = 0.00000002*(exp(PHI*105.56))
        elif tops == 'Balakhany_VIII_sand' or tops == 'Balakhany VIII sand':
            PERM = 0.00000002*(exp(PHI*105.56))
        else:
            #PERM = 0.0000000006*(exp(PHI*128.04)) Old balX transform,
            PERM = 0.00000002*(exp(PHI*105.56))
            
    elif PHI >= 0.15 and PHI < 0.16:#EQ2
        if  tops == 'Pereriv_D' or tops == 'Pereriv D':
            PERM =  0.0000029157*(exp(PHI*89.8991405))
        elif  tops == 'Pereriv_D_lower' or tops == 'Pereriv D lower':
            PERM =  0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'Pereriv_E' or tops == 'Pereriv E':
            PERM =  0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'Pereriv_A' or tops == 'Pereriv A':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'Pereriv_B' or tops == 'Pereriv B':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'NKG':
            PERM = 0.0000029157*(exp(PHI*89.8991405))
        elif tops == 'Pereriv_C' or tops == 'Pereriv C':
            PERM = 0.0293*(exp(PHI*22.07))
        elif tops == 'Balakhany_VIII' or tops == 'Balakhany VIII':
            PERM = 0.0159*(exp(PHI*21.27))
        elif tops == 'Balakhany_VIII_sand' or tops == 'Balakhany VIII sand':
            PERM = 0.0159*(exp(PHI*21.27))
        else:
            PERM = 0.0159*(exp(PHI*21.27))
    elif PHI == -9999:
        PERM = -9999
    else:
        if  tops == 'Pereriv_D' or tops == 'Pereriv D':
            PERM = 0.055*(exp(PHI*24.43))
        elif  tops == 'Pereriv_D_lower' or tops == 'Pereriv D lower':
            PERM = 0.055*(exp(PHI*24.43))
        elif tops == 'Pereriv_E' or tops == 'Pereriv E':
            PERM = 0.055*(exp(PHI*24.43))
        elif tops == 'Pereriv_A' or tops == 'Pereriv A':
            PERM = 0.055*(exp(PHI*24.43))
        elif tops == 'Pereriv_B' or tops == 'Pereriv B':
            PERM = 0.055*(exp(PHI*24.43))
        elif tops == 'NKG':
            PERM = 0.055*(exp(PHI*24.43))
        elif tops == 'Pereriv_C' or tops == 'Pereriv C':
            PERM = 0.0293*(exp(PHI*22.07))
        elif tops == 'Balakhany_VIII' or tops == 'Balakhany VIII':
            PERM = 0.0159*(exp(PHI*21.27))
        elif tops == 'Balakhany_VIII_sand' or tops == 'Balakhany VIII sand':
            PERM = 0.0159*(exp(PHI*21.27))
        else:
            PERM = 0.0159*(exp(PHI*21.27))
            
            
    return PERM


def calculate_KH(well,suffix):
    
    well = well.copy()
    
    new_LPERMs = []
    for (indx,row) in well.iterrows():
        
        perm = 0
        
        if row["NET_clp2"] == 1:
            perm = getPerm(row[f"PHIT_{suffix}"],row["FORMATION"])
            
        new_LPERMs.append(perm)
        
    well[f"LPERM_{suffix}"] = new_LPERMs
    well[f"KHtst_{suffix}"] =  well[f"LPERM_{suffix}"] * 0.1
    well[f"KHtst_{suffix}_sum"] = well.loc[::-1, f"KHtst_{suffix}"].cumsum()[::-1]
    
    
    return well



class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 256, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(256, 128, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(128, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(64, 128, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.ConvTranspose1d(128, 256, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.ConvTranspose1d(256, 1, kernel_size=5, stride=1, padding=0),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



def slice_dataframe(df, col_selection1="VSH", col_selection2="PHIT", chunk_size=100, stride=100):
    selection1_slices = []
    selection2_slices = []
    general_slice_data = []

    start_index = 0

    while start_index < len(df):
        end_index = start_index + chunk_size
        slice_data = df.iloc[start_index:end_index]

        if len(slice_data) == chunk_size:
            selection1_slices.append(slice_data[col_selection1].values.flatten())
            selection2_slices.append(slice_data[col_selection2].values.flatten())
            general_slice_data.append(slice_data)

        start_index += stride

    return selection1_slices, selection2_slices, general_slice_data




def slice_dataframe_multiple(df, col_selection1=["X","Y","TST","VSH"], col_selection2="PHIT", chunk_size=100, stride=100):
    selection1_slices = []
    selection2_slices = []
    general_slice_data = []

    start_index = 0

    while start_index < len(df):
        end_index = start_index + chunk_size
        slice_data = df.iloc[start_index:end_index]

        if len(slice_data) == chunk_size:
        
            selection1_slices.append(slice_data[col_selection1].values.T)
            selection2_slices.append(slice_data[col_selection2].values.flatten())
            general_slice_data.append(slice_data)

        start_index += stride

    return selection1_slices, selection2_slices, general_slice_data







def seeding(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True




def normalize_column(dataframe, column_name):
    """
    Normalize a specific column in a DataFrame using MinMaxScaler.

    Parameters:
    - dataframe: pandas DataFrame
    - column_name: str, the name of the column to be normalized

    Returns:
    - normalized_column: pandas Series, the normalized column
    - scaler: MinMaxScaler, the scaler used for normalization
    """
    scaler = MinMaxScaler()
    column_data = dataframe[column_name].values.reshape(-1, 1)
    normalized_column = scaler.fit_transform(column_data).flatten()
    return pd.Series(normalized_column, name=column_name + '_normalized'), scaler





def denormalize_column(normalized_column, scaler):
    """
    Denormalize a column using the provided MinMaxScaler.

    Parameters:
    - normalized_column: pandas Series, the normalized column to be denormalized
    - scaler: MinMaxScaler, the scaler used for normalization

    Returns:
    - denormalized_column: pandas Series, the denormalized column
    """
    normalized_data = normalized_column.reshape(-1, 1)
    denormalized_column = scaler.inverse_transform(normalized_data).flatten()
    return denormalized_column





# dataset = pd.read_parquet("data/ACG_wells_JOINT_BEST_v10.parquet.gzip")
# dataset.rename(columns={'wellName': 'well'}, inplace=True)

dataset = pd.read_csv("df_bal_net2_kh.csv")
dataset = dataset[["well","FORMATION","FORMATION_up","PHIT","VSH","GR_N","LPERM","NET_clp2","KHtst","TVD_SCS","TST","X","Y"]]
valid_wells = pd.read_csv('well_clean_v2_df.csv')
valid_wells = valid_wells.groupby('FORMATION_up')['well'].agg(list).to_dict()

well_names_to_exclude = sorted(list(set(valid_wells["Balakhany VIII"]).union(set(valid_wells["Balakhany X"]))))
# well_names_to_exclude = valid_wells["Balakhany X"]

normalized_phit , phit_scaler = normalize_column(dataset[(dataset["PHIT"] >= 0.05) & (dataset["PHIT"] <= 0.3)],"PHIT")


for exclude_well in well_names_to_exclude:
    
    print(exclude_well)
    
    well_names = list(dataset["well"].unique())
    formations = dataset['FORMATION_up'].unique()
#     formations = ["Balakhany X"]
    all_formations_smoothings = []

    X = []
    y = []
    general = []

    for i,formation in enumerate(formations):
        print(formation)


        current_dataset = dataset[dataset['FORMATION_up'].str.contains(formation)]


        current_well_names = list(current_dataset["well"].unique())


        for name in valid_wells[formation]:

            if name == exclude_well:
                continue


            one_well = current_dataset[current_dataset["well"]==name].copy()

            one_well = interpolate_by_depth(one_well,"TST")


            one_well["PHIT_normalized"] = phit_scaler.transform(one_well["PHIT"].values.reshape(-1, 1)).flatten()

 
            VSH_slices , phit_slices, general_slices = slice_dataframe_multiple(one_well, col_selection1 ="VSH", col_selection2 = "PHIT_normalized" ,chunk_size=100,stride=10)


            X = X + VSH_slices
            y = y + phit_slices

            general = general + general_slices



    X = np.array(X)
    y = np.array(y)


    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=1234)


    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
    Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)

    train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)



    num_epochs = 200
    learning_rate = 0.0001

    seeding(1234)
    

    model = Autoencoder()

    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    best_val_loss = float('inf')
    best_model_state = None

    model.train()
    for epoch in range(num_epochs):
        total_train_loss = 0

        for batch_x, batch_y in train_dataloader:
            batch_x, batch_y = batch_x.unsqueeze(1).to(device), batch_y.unsqueeze(1).to(device)
            optimizer.zero_grad()
            out = model(batch_x)
            loss = criterion(out.view(-1), batch_y.view(-1))
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch_x, batch_y in test_dataloader:
                batch_x, batch_y = batch_x.unsqueeze(1).to(device), batch_y.unsqueeze(1).to(device)
                out = model(batch_x)
                loss = criterion(out.view(-1), batch_y.view(-1))
                total_val_loss += loss.item()

        average_train_loss = total_train_loss / len(train_dataloader)
        average_val_loss = total_val_loss / len(test_dataloader)

        print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_train_loss:.8f}, Validation Loss: {average_val_loss:.8f}')

        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss
            best_model_state = model.state_dict()
            checkpoint_path = f'Base/{exclude_well}_base_autoencoder_checkpoint.pth'
            torch.save({'epoch': epoch + 1, 'model_state_dict': best_model_state, 'optimizer_state_dict': optimizer.state_dict(), 'val_loss': best_val_loss}, checkpoint_path)
            print(f'Saved checkpoint at epoch {epoch + 1} with validation loss: {best_val_loss:.8f}')

    print("Training complete.")


    for i,formation in enumerate(formations):
        print(formation)


        current_dataset = dataset[dataset['FORMATION_up'].str.contains(formation)]


        current_well_names = list(current_dataset["well"].unique())


        for name in valid_wells[formation]:

            if name != exclude_well:
                continue


            one_well = current_dataset[current_dataset["well"]==name].copy()

        one_well = interpolate_by_depth(one_well,"TST")


        one_well["PHIT_normalized"] = phit_scaler.transform(one_well["PHIT"].values.reshape(-1, 1)).flatten()
        

        VSH_slices , phit_slices, general_slices = slice_dataframe_multiple(one_well, col_selection1 ="VSH", col_selection2 = "PHIT_normalized" ,chunk_size=100,stride=100)

        VSH_slices = np.array(VSH_slices)
        phit_slices = np.array(phit_slices)


        X_tensor = torch.tensor(VSH_slices, dtype=torch.float32)
        Y_tensor = torch.tensor(phit_slices, dtype=torch.float32)

        dataset_ = TensorDataset(X_tensor, Y_tensor)
        dataloader = DataLoader(dataset_, batch_size=1, shuffle=False)


        model.eval()

        predictions = []

        with torch.no_grad():
            for batch_x, _ in dataloader:
                batch_x = batch_x.unsqueeze(1).to(device)
                out = model(batch_x)
                predictions.append(out.cpu().numpy().flatten())

        predictions = np.concatenate(predictions, axis=0)

        predictions = denormalize_column(predictions, phit_scaler)

        one_well = one_well.iloc[:len(predictions),:]
        one_well["PHIT_predicted"] = predictions
        one_well = calculate_KH(one_well,"predicted")

        print("Original KH: ",one_well["KHtst"].dropna().max())
        print("Predicted KH: ",one_well["KHtst_predicted_sum"].dropna().max())

        phit_slices = phit_slices.flatten()
        phit_slices = denormalize_column(phit_slices, phit_scaler)


        plt.figure(figsize=(6, 12))
        plt.plot(phit_slices,[i for i in range(len(predictions))], label='Original X', linestyle='--',markersize=3)
        plt.plot(predictions,[i for i in range(len(predictions))], label='Predicted X', linestyle='--',markersize=3)
        plt.axvline(x=0.13, color='red', linestyle='--', label='PHIT = 0.13')


        plt.title(f'{exclude_well} ({formation}) Original KH: {one_well["KHtst"].dropna().max():.1f} | Predicted KH: {one_well["KHtst_predicted_sum"].dropna().max():.1f}')
        plt.gca().invert_yaxis()
        plt.legend()
        
        plt.savefig(f'Base/{exclude_well}_{formation}_base.png')
        plt.show()
    